In [28]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [29]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [30]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [31]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [32]:

#  CREATE GRAPH
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, BC19GBtds8YpcsC5kahXnKdaACxr59XNBKhCWuDv9jG4, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [33]:

#  Create Vertices/Nodes ..

df_airports = pd.DataFrame([
       #
   ["MKE" , "Milwaukee"               , "Airport"],
   ["ORD" , "Chicago O-Hare"          , "Airport"],
       #
   ], columns = ["id", "airport_name", "LABEL"])
      #
print(tabulate(df_airports, headers='keys', tablefmt='psql'))

df_persons = pd.DataFrame([
       #
   ["BOB" , "Bob"                     , "Person" ],
   ["HADI", "Hadi"                    , "Person" ],
       #
   ], columns = ["id", "person_name" , "LABEL"])
      #
print(tabulate(df_persons, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+------+----------------+---------+
#     |    | id   | airport_name   | LABEL   |
#     |----+------+----------------+---------|
#     |  0 | MKE  | Milwaukee      | Airport |
#     |  1 | ORD  | Chicago O-Hare | Airport |
#     +----+------+----------------+---------+
#     +----+------+---------------+---------+
#     |    | id   | person_name   | LABEL   |
#     |----+------+---------------+---------|
#     |  0 | Bob  | Bob           | Person  |
#     |  1 | Hadi | Hadi          | Person  |
#     +----+------+---------------+---------+


+----+------+----------------+---------+
|    | id   | airport_name   | LABEL   |
|----+------+----------------+---------|
|  0 | MKE  | Milwaukee      | Airport |
|  1 | ORD  | Chicago O-Hare | Airport |
+----+------+----------------+---------+
+----+------+---------------+---------+
|    | id   | person_name   | LABEL   |
|----+------+---------------+---------|
|  0 | BOB  | Bob           | Person  |
|  1 | HADI | Hadi          | Person  |
+----+------+---------------+---------+


In [34]:

#  Create the Edge/Relationships ..

df_relates_to_N = pd.DataFrame([
      #
   ["BOB" , "ORD" , "RELATES_TO"],
      #
   ["HADI", "MKE" , "RELATES_TO"],
   ["HADI", "ORD" , "RELATES_TO"],
      #
   ], columns = ["START_ID", "END_ID", "TYPE"])
      #
print(tabulate(df_relates_to_N, headers='keys', tablefmt='psql'))

df_relates_to_S = pd.DataFrame([
      #
   ["ORD" , "BOB" , "RELATES_TO"],
      #
   ["MKE" , "HADI", "RELATES_TO"],
      #
   ], columns = ["START_ID", "END_ID", "TYPE"])
      #
print(tabulate(df_relates_to_S, headers='keys', tablefmt='psql'))

#  Sample output,
#
#      +----+------------+----------+------------+
#      |    | START_ID   | END_ID   | TYPE       |
#      |----+------------+----------+------------|
#      |  0 | BOB        | ORD      | RELATES_TO |
#      |  1 | HADI       | MKE      | RELATES_TO |
#      |  2 | HADI       | ORD      | RELATES_TO |
#      +----+------------+----------+------------+
#      +----+------------+----------+------------+
#      |    | START_ID   | END_ID   | TYPE       |
#      |----+------------+----------+------------|
#      |  0 | ORD        | BOB      | RELATES_TO |
#      |  1 | MKE        | HADI     | RELATES_TO |
#      +----+------------+----------+------------+


+----+------------+----------+------------+
|    | START_ID   | END_ID   | TYPE       |
|----+------------+----------+------------|
|  0 | BOB        | ORD      | RELATES_TO |
|  1 | HADI       | MKE      | RELATES_TO |
|  2 | HADI       | ORD      | RELATES_TO |
+----+------------+----------+------------+
+----+------------+----------+------------+
|    | START_ID   | END_ID   | TYPE       |
|----+------------+----------+------------|
|  0 | ORD        | BOB      | RELATES_TO |
|  1 | MKE        | HADI     | RELATES_TO |
+----+------------+----------+------------+


In [ ]:

#  Import the previously created Python PandasDataFrames into KatanaGraph ..


with import_data.DataFrameImporter(my_graph) as df_importer:   
   df_importer.nodes_dataframe(
      df_airports,
      id_column             = "id",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   df_importer.insert()
    
with import_data.DataFrameImporter(my_graph) as df_importer:   
   df_importer.nodes_dataframe(
      df_persons,
      id_column             = "id",
      id_space              = "Person",  
      label                 = "Person",  
      )
   df_importer.insert()
    
    
with import_data.DataFrameImporter(my_graph) as df_importer:   
   df_importer.edges_dataframe(
      df_relates_to_N,
      source_id_space       = "Airport", 
      destination_id_space  = "Person",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "RELATES_TO"
      )
         #
   df_importer.node_id_property_name("id")
      #
   df_importer.insert()

with import_data.DataFrameImporter(my_graph) as df_importer:   
   df_importer.edges_dataframe(
      df_relates_to_S,
      source_id_space       = "Person", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "RELATES_TO"
      )
         #
   df_importer.node_id_property_name("id")
      #
   df_importer.insert()

print("--")



          0/? [?op/s]

          0/? [?op/s]

          0/? [?op/s]

          0/? [?op/s]

          0/? [?op/s]

#  Verifying Results ..

In [26]:

display(my_graph.num_nodes())
display(my_graph.num_edges())

#  Sample output,
#


          0/? [?op/s]

0

          0/? [?op/s]

0

In [27]:

#  What's in the graph ..

l_result = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)
print(tabulate(l_result, headers='keys', tablefmt='psql'))


          0/? [?op/s]